In [49]:
import pandas as pd
import numpy as np
import warnings
import time
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
conj_to_predict = pd.read_csv('../csvs/compounds30_ni_29.csv', index_col = 0)
top3 = pd.read_csv('../csvs/comp_sim_df_v30.csv', index_col=0)
perfil_afinidad_activity = pd.read_csv('../csvs/perfil_afinidad_celulas_29_plusActivity, Obtener perfiles de afinidad.csv')

In [3]:
conj_to_predict = conj_to_predict.sort_values('compound_id')
conj_to_predict = conj_to_predict.assign(prediccion='')
conj_to_predict.reset_index(drop=True, inplace=True)

In [4]:
#esta función retorna la lista de compuestos que interactuan con una celula, además de la actividad
def get_listaCompuestos_conActividad(cell):
    try:
        lista_compuestos_actividad = 
        [eval(perfil_afinidad_activity['id_compuestos_actividad'][perfil_afinidad_activity['cell_id']==cell].tolist()[0]),
                        eval(perfil_afinidad_activity['activity'][perfil_afinidad_activity['cell_id']==cell].tolist()[0])]
    except:
        lista_compuestos_actividad = []
    return lista_compuestos_actividad

In [5]:
#esta función retorna los compuestos (en forma de lista) pertenecientes al top3 del compuesto que se envia como parámetro
def get_top3_compound(compuesto):
    top3_compuesto = []
    try:
        top3_compuesto = eval(top3['top3'][top3['comp_id'] == compuesto].tolist()[0])
    except:
        pass
    return top3_compuesto

In [6]:
#Esta función retorna el valor de la prediccion en 
def return_prediccion(array_prediccion):
    prediccion_final = 0
    resultado = all(element == array_prediccion[0] for element in array_prediccion)
    if (resultado):
#         print("todos los elementos iguales")
        prediccion_final = array_prediccion[0]
    else:
#         print("no son iguales")
        prediccion_final = 0        
    return prediccion_final

In [ ]:
for i in conj_to_predict.index: #Recorro el conjunto de datos a predecir
    array_interaccion = []
    compuesto_a_predecir = conj_to_predict['compound_id'][i] #Par compuesto - línea celular al que se 
    linea_celular_a_predecir = conj_to_predict['cell_id'][i] #le va a realizar la predicción
    perfil_afinidad = get_listaCompuestos_conActividad(linea_celular_a_predecir) #Obtención del perfil de afinidad
    if perfil_afinidad:
        for j in perfil_afinidad[0]:
            top3_cada_compuesto = get_top3_compound(j)#Obtención del top3 de cada compuesto del perfil de afinidad
            if top3_cada_compuesto:
                if compuesto_a_predecir in top3_cada_compuesto:
                    array_interaccion.append(perfil_afinidad[0].index(j))#Aquí se guarda el índice del valor de la interacción
    else:
        conj_to_predict.at[i, 'prediccion'] = 'NA' #En caso de que no exista un perfil de afinidad para la línea celular
        
    if len(array_interaccion) > 0:
        if len(array_interaccion) == 1: #Si existe solo un valor de interacción, se coloca ese valor en la predicción
            conj_to_predict.at[i, 'prediccion'] = perfil_afinidad[1][array_interaccion[0]]
        elif len(array_interaccion) > 1: #Se verifica que todos los valores de interacción sean iguales
            resultado = all(element == array_interaccion[0] for element in array_interaccion)
            if (resultado): #En caso de que todos los valores sean iguales
                conj_to_predict.at[i, 'prediccion'] = perfil_afinidad[1][array_interaccion[0]]
            else: #En caso de que no todos los valores sean iguales
                conj_to_predict.at[i, 'prediccion'] = 0     

In [7]:
#tercer algoritmo
start = time.time()
for i in conj_to_predict.index:
    indice_en_lista = []
    prediccion  = []
    compuesto_a_predecir = conj_to_predict['compound_id'][i]
    linea_celular_a_predecir = conj_to_predict['cell_id'][i]
    lc = get_listaCompuestos_conActividad(linea_celular_a_predecir)
    if lc:
        for j in lc[0]:
            top3cc = get_top3_compound(j)
            if top3cc:
                if compuesto_a_predecir in top3cc:
                    indice_en_lista.append(lc[0].index(j))
    else:
        conj_to_predict.at[i, 'prediccion'] = 'NA'
        
    if len(indice_en_lista) > 0:
        if len(indice_en_lista) == 1:
            conj_to_predict.at[i, 'prediccion'] = lc[1][indice_en_lista[0]]
        elif len(indice_en_lista) > 1:
            resultado = all(element == indice_en_lista[0] for element in indice_en_lista)
            if (resultado):
                conj_to_predict.at[i, 'prediccion'] = lc[1][indice_en_lista[0]]
            else:
                conj_to_predict.at[i, 'prediccion'] = 0       
                
end = time.time()
print(end - start)

10411.428629159927


In [48]:
nueva_pred = conj_to_predict
nueva_pred
# nueva_pred[nueva_pred['active'] == -1 & (nueva_pred['prediccion'] == 1)]
# nueva_pred.loc[(nueva_pred['prediccion'] != -1) | (nueva_pred['prediccion'] != 1) | (nueva_pred['prediccion'] != 0) | (nueva_pred['prediccion'] != 'NA')]
# nueva_pred.loc[(nueva_pred['prediccion'] == -1) | (nueva_pred['prediccion'] == 1) | (nueva_pred['prediccion'] == 0)]

,compound_id,cell_id,active,prediccion
0,97,5673,-1,NA
1,115,5673,-1,NA
2,146,5673,-1,NA
3,147,5673,-1,NA
4,148,325,-1,NA
...,...,...,...,...
66309,2537375,5673,-1,NA
66310,2537376,303,-1,NA
66311,2537376,721,-1,NA
66312,2537376,786,-1,NA


In [44]:
# nueva_pred['prediccion'] = nueva_pred['prediccion'].replace([''],'NA')
predicciones = nueva_pred[nueva_pred['prediccion'] != 'NA' ]
predicciones.reset_index(drop=True, inplace=True)
predicciones

,compound_id,cell_id,active,prediccion
0,561,491,-1,-1
1,594,5673,-1,-1
2,853,377,-1,-1
3,1114,704,-1,-1
4,1114,5737,-1,0
...,...,...,...,...
579,2536958,588,-1,-1
580,2537014,786,-1,-1
581,2537014,491,-1,-1
582,2537014,646,-1,-1


In [45]:
predicciones.to_csv('../csvs/nueva_pred_alg3_SR_Final.csv')

In [111]:
# 594	5673
# 561	491
lc = get_listaCompuestos_conActividad(5673)
indice_en_lista = []
prediccion  = []
cont_casos = 0
for i in lc[0]:
    top3ccc = get_top3_compound(i)
    if top3ccc:
        if 594 in top3ccc:
            cont_casos = cont_casos + 1
            indice_en_lista.append(lc[0].index(i))
print("indice en lista", indice_en_lista)
if len(indice_en_lista) > 0:
    if len(indice_en_lista) == 1:
        test = lc[1][indice_en_lista[0]]
#             conj_to_predict.at[i, 'prediccion'] = lc[1][indice_en_lista[0]]
    elif len(indice_en_lista) > 1:
        resultado = all(element == indice_en_lista[0] for element in indice_en_lista)
        if (resultado):
            test = lc[1][indice_en_lista[0]]
#                 conj_to_predict.at[i, 'prediccion'] = lc[1][indice_en_lista[0]]
        else:
            test = 0        
#                 conj_to_predict.at[i, 'prediccion'] = 0 
test

indice en lista [11]


-1